# How to use this notebook (recommended for non-CLI users)

This notebook is designed for users who prefer working interactively rather than running command-line scripts.

- Step 1: Ensure you have activated the `pyagri-notebooks` environment and installed the package in editable mode (`pip install -e .`). See the top cell for environment checks.
- Step 2: Set the `PROJECT_ROOT` variable in the **Configuration** cell if you want to use an explicit absolute path; otherwise the notebook will use the current working directory. All other data paths (inputs and outputs) are relative to `PROJECT_ROOT`.
- Step 3: In the **Configuration** cell, confirm `INPUT_TASKDATA` (list of TaskData folders), `LEDREBORG_OUT` and `HARVEST_GEOJSON` look correct.
- Step 4 (recommended): Run the small **Per-input API** cell for the specific input(s) you want to process — this calls the library functions directly and shows concise results inline.
- Step 5: Use the **Advanced: Batch run** cell only if you want to process multiple inputs in one go (this runs subprocesses and is better for batch jobs, not interactive exploration).

If a GeoJSON or CSV already exists, the notebook will append features or rows rather than overwrite, so you can safely run the same input multiple times for incremental exports.

Tip: to quickly set `PROJECT_ROOT` to the workspace root use:

```python
PROJECT_ROOT = Path(r"C:/dev/agri_analysis").resolve()
```

In [1]:
# 01_Raw_read
# Friendly notebook that reproduces the CLI workflows:
#  - python -m pyagri.export data/TASKDATA_2 data/ledreborg_CSV
#  - python -m pyagri.geo data/TASKDATA data/harvest_fields_s.geojson

# NOTE: This notebook is intended to be run in the `pyagri-notebooks` environment
# (or another environment with the project installed in editable mode).
# Recommended quick setup (run in a shell once):
#   micromamba activate pyagri-notebooks
#   pip install -e .
#   pip install geopandas pandas

# Section 1: Quick environment check
import sys
from pathlib import Path
import logging

logger = logging.getLogger('01_Raw_read')
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
logger.addHandler(handler)

logger.info('Python executable: %s', sys.executable)
try:
    import pyagri
    logger.info('pyagri import OK')
except Exception as e:
    logger.warning('pyagri import failed: %s', e)


Python executable: /Users/holmes/micromamba/envs/pyagri-notebooks/bin/python
pyagri import OK


In [2]:
# Configuration cell — set project root, inputs and outputs here
# Set the PROJECT_ROOT to the absolute path of the folder containing the data (the previous cell set it to the folder of the notebook)
# Example explicit override (uncomment and adjust if you run the notebook from a different working directory):
PROJECT_ROOT = Path.cwd().resolve().parent

# PROJECT_ROOT = Path(r"C:/dev/agri_analysis").resolve()

DATA_DIR = PROJECT_ROOT / 'data'

# You can specify more than one TaskData folder in INPUT_TASKDATA list. These are paths relative to PROJECT_ROOT.
INPUT_TASKDATA = [
    DATA_DIR / 'TASKDATA_2',  # example input folder 1
    DATA_DIR / 'TASKDATA',  # add more as needed
]

# CSV output directory (one dir used for all inputs; files for each task will be appended)
HARVESTER_POINTS_DIR = PROJECT_ROOT / 'data' / 'ledreborg_CSV'
HARVESTER_POINTS_DIR.mkdir(parents=True, exist_ok=True)

# GeoJSON output file for task polygons (appends features if file exists)
FIELD_POLYGONS_GEOJSON = PROJECT_ROOT / 'data' / 'harvest_fields_s.geojson'

# --- Set target projected CRS for area/overlap analysis ---
# Use EPSG:25832 (ETRS89 / UTM zone 32N) for Denmark
EPSG = '25832'  # Change as needed for your region


logger.info('Project root: %s', PROJECT_ROOT)
logger.info('Configured %d input folders: %s', len(INPUT_TASKDATA), [str(p) for p in INPUT_TASKDATA])
logger.info('CSV output dir: %s', HARVESTER_POINTS_DIR)
logger.info('GeoJSON output file: %s', FIELD_POLYGONS_GEOJSON)
logger.info('Target projected CRS EPSG: %s', EPSG)


Project root: /Users/holmes/local_dev/agri_analysis
Configured 2 input folders: ['/Users/holmes/local_dev/agri_analysis/data/TASKDATA_2', '/Users/holmes/local_dev/agri_analysis/data/TASKDATA']
CSV output dir: /Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV
GeoJSON output file: /Users/holmes/local_dev/agri_analysis/data/harvest_fields_s.geojson
Target projected CRS EPSG: 25832


## Loding binary data
The following cell loads the data form the XML and binary data and exports it to CSV and GeoJSON for futher analasis. If you alredy have converted the data and only want to validate the data skip the following cell

In [3]:
# Per-input API example 
try:
    from pyagri import export as export_mod
    from pyagri import geo as geo_mod
    for src in INPUT_TASKDATA:
        srcp = Path(src)
        if not srcp.exists():
            logger.warning('Skipping missing input: %s', srcp)
            continue
        logger.info('Exporting (API) for: %s', srcp)
        written = export_mod.export_taskdata(str(srcp), str(HARVESTER_POINTS_DIR))
        logger.info('Wrote CSV files: %s', written)

        logger.info('Extracting GeoJSON (API) for: %s', srcp)
        rc = geo_mod.extract_taskdata_to_geojson(str(srcp), str(FIELD_POLYGONS_GEOJSON))
        logger.info('Geo extraction returned: %s (%s)', rc, 'success' if rc == 0 else 'error')
except Exception as e:
    logger.exception('Per-input API example failed: %s', e)


Skipping missing input: /Users/holmes/local_dev/agri_analysis/data/TASKDATA_2
Exporting (API) for: /Users/holmes/local_dev/agri_analysis/data/TASKDATA
Wrote CSV files: ['/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/021-0_ Monica.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/021-0_ Monica.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/021-0_ Monica.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/Import.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/022-0_ Lidia.csv', '/Users/holmes/local_dev/agri_analysis/data/ledreborg_CSV/022-0_ Lidia.csv', '/Users/holmes/lo

In [ ]:
# --- Display enhanced TASKDATA.XML report (with crop, TLG, sensors) ---
from pyagri.taskdata_report import taskdata_report

xml_path = PROJECT_ROOT / 'data' / 'taskdata' / 'TASKDATA.XML'
taskdata_report(xml_path)


In [ ]:
import importlib
import pyagri.taskdata_report
importlib.reload(pyagri.taskdata_report)
from pyagri.taskdata_report import taskdata_report_df

In [ ]:
# --- Display TASKDATA.XML report as a DataFrame ---
from pyagri.taskdata_report import taskdata_report_df
from IPython.display import display

xml_path = PROJECT_ROOT / 'data' / 'taskdata' / 'TASKDATA.XML'
df_report = taskdata_report_df(xml_path)
display(df_report)


## Premlmary data analasis
The data is now loaded and the following cells contain simpel valdation of the input

In [ ]:
# --- GeoJSON Year Investigation ---
import geopandas as gpd
import pandas as pd
from IPython.display import display

# Load the GeoJSON file
gdf = gpd.read_file(FIELD_POLYGONS_GEOJSON)

# List all unique years in the 'Year' attribute
years = sorted(gdf['Year'].dropna().unique())
print(f"Unique years in GeoJSON: {years}")
display(pd.DataFrame({'Year': years}))

In [ ]:

# Reproject GeoDataFrame to the target CRS for analysis
gdf_proj = gdf.to_crs(epsg=EPSG)
print(f"Reprojected GeoDataFrame to EPSG:{EPSG} for area/overlap analysis.")


In [ ]:
# --- Check for overlapping polygons within each year (projected CRS) ---
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Function to find overlapping polygons in a GeoDataFrame for a given year
def find_overlaps(gdf_year):
    overlaps = []
    for i, row1 in gdf_year.iterrows():
        for j, row2 in gdf_year.iterrows():
            if i >= j:
                continue
            if row1['geometry'].intersects(row2['geometry']):
                intersection = row1['geometry'].intersection(row2['geometry'])
                if not intersection.is_empty and intersection.area > 0:
                    overlaps.append((i, j, intersection.area))
    return overlaps

# Analyze overlaps for each year using the projected GeoDataFrame
overlap_summary = {}
for year in years:
    gdf_year = gdf_proj[gdf_proj['Year'] == year]
    overlaps = find_overlaps(gdf_year)
    overlap_summary[year] = overlaps
    print(f"Year {year}: {len(overlaps)} overlapping pairs found.")
    if overlaps:
        for i, j, area in overlaps:
            print(f"  Overlap between index {i} and {j}, area: {area:.2f} m²")

# Optionally, display summary as a DataFrame
overlap_counts = pd.DataFrame({
    'Year': list(overlap_summary.keys()),
    'Num Overlaps': [len(v) for v in overlap_summary.values()]
})
display(overlap_counts)

In [7]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from datetime import datetime

def format_iso_time(iso_str):
    """Converts ISO 8601 string to human-readable 'YYYY-MM-DD HH:MM:SS'"""
    if not iso_str:
        return "Unknown"
    try:
        # Handle string slicing for simple conversion to avoid timezone library complexity
        # Expecting format like: 2023-08-16T12:39:28.000+02:00
        dt_str = iso_str.split('.')[0] # Remove milliseconds and timezone for display
        return dt_str.replace('T', ' ')
    except:
        return iso_str

def parse_isoxml_taskdata(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # --- 1. Extract Farm & Customer ---
    farms = {}
    for frm in root.findall('FRM'):
        farms[frm.attrib.get('A')] = frm.attrib.get('B', 'Unknown Farm')

    # --- 2. Extract Fields (PFD) ---
    fields = {}
    for pfd in root.findall('PFD'):
        pfd_id = pfd.attrib.get('A')
        name = pfd.attrib.get('C') or pfd.attrib.get('B') or f"Field {pfd_id}"
        fields[pfd_id] = name

    # --- 3. Extract Products/Crops (PDT) ---
    products = {}
    for pdt in root.findall('PDT'):
        products[pdt.attrib.get('A')] = pdt.attrib.get('B', 'Unknown Product')

    # --- 4. Extract Device Data (DVC, DET, DPD, DVP) ---
    # Map DET -> DVC, and store DPD/DVP definitions
    det_to_dvc = {}
    dvc_info = {}

    for dvc in root.findall('DVC'):
        dvc_id = dvc.attrib.get('A')
        dvc_name = dvc.attrib.get('B', 'Unknown Machine')
        
        for det in dvc.findall('.//DET'):
            det_to_dvc[det.attrib.get('A')] = dvc_id

        # Parse DVP (Value Presentation)
        dvp_map = {}
        for dvp in dvc.findall('.//DVP'):
            try:
                dvp_id = dvp.attrib.get('A')
                dvp_map[dvp_id] = {
                    'scale': float(dvp.attrib.get('C', 1)),
                    'offset': float(dvp.attrib.get('B', 0)),
                    'unit': dvp.attrib.get('E', '')
                }
            except: continue

        # Parse DPD (Process Data)
        dpd_map = {}
        for dpd in dvc.findall('.//DPD'):
            ddi = dpd.attrib.get('B')
            prop_name = dpd.attrib.get('E', f"DDI_{ddi}")
            dvp_ref = dpd.attrib.get('F')
            dpd_map[ddi] = {'name': prop_name, 'dvp': dvp_map.get(dvp_ref)}
            
        dvc_info[dvc_id] = {'name': dvc_name, 'dpds': dpd_map}

    # --- 5. Process Tasks (TSK) ---
    report_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    for tsk in root.findall('TSK'):
        tsk_id = tsk.attrib.get('A')
        field_ref = tsk.attrib.get('E')
        
        field_name = fields.get(field_ref, f"Unknown Field ({field_ref})")

        # Crops
        task_crops = [products[pan.attrib.get('A')] for pan in tsk.findall('PAN') if pan.attrib.get('A') in products]
        crop_name = ", ".join(task_crops) if task_crops else "None"

        # Machine
        dan = tsk.find('DAN')
        machine_name = "Generic Machine"
        dvc_id_for_task = None
        if dan is not None:
            dvc_id_for_task = dan.attrib.get('C')
            if dvc_id_for_task in dvc_info:
                machine_name = dvc_info[dvc_id_for_task]['name']

        # TLG Files
        tlg_files = [tlg.attrib.get('A') for tlg in tsk.findall('TLG')]

        # --- Process Time & Values ---
        start_times = []
        end_times = []
        logged_values = []
        
        for tim in tsk.findall('TIM'):
            if 'A' in tim.attrib: start_times.append(tim.attrib['A'])
            if 'B' in tim.attrib: end_times.append(tim.attrib['B'])

            for dlv in tim.findall('DLV'):
                ddi = dlv.attrib.get('A')
                raw_value = int(dlv.attrib.get('B', 0))
                det_ref = dlv.attrib.get('C')
                
                # Default property info
                prop_name = f"DDI {ddi}"
                final_val = raw_value
                unit = ""
                
                # 1. Check Hardcoded Overrides (The "User Request" Layer)
                if ddi == '0106': # Fugtighed (Moisture)
                    prop_name = "Fugtighed"
                    final_val = raw_value * 0.0001
                    unit = "%"
                elif ddi == '013B': # Tørstofindhold (Dry Matter)
                    prop_name = "Tørstofindhold"
                    final_val = raw_value * 0.0001
                    unit = "%"
                
                # 2. If not overridden, look up DVP definitions in XML
                else:
                    current_dvc_id = det_to_dvc.get(det_ref, dvc_id_for_task)
                    if current_dvc_id and current_dvc_id in dvc_info:
                        dvc_defs = dvc_info[current_dvc_id]['dpds']
                        if ddi in dvc_defs:
                            def_info = dvc_defs[ddi]
                            prop_name = def_info['name']
                            if def_info['dvp']:
                                dvp = def_info['dvp']
                                final_val = (raw_value * dvp['scale']) + dvp['offset']
                                unit = dvp['unit']
                            # Standard fallback units if XML is missing DVP
                            elif ddi == '0074': unit = "m²"
                            elif ddi == '005A': unit = "kg"

                # Formatting
                if isinstance(final_val, float):
                    val_str = f"{final_val:,.2f}"
                else:
                    val_str = str(final_val)

                entry = f"{prop_name}: {val_str} {unit}"
                if entry not in logged_values:
                    logged_values.append(entry)

        # Determine overall start/end
        start_ts = min(start_times) if start_times else None
        end_ts = max(end_times) if end_times else None
        
        # Determine Year
        year = start_ts[:4] if start_ts else "Unknown Year"
        
        # Formatting Time
        time_range = f"{format_iso_time(start_ts)} - {format_iso_time(end_ts)}"

        task_info = {
            'TaskID': tsk_id,
            'Machine': machine_name,
            'Crop': crop_name,
            'TimeRange': time_range,
            'TLGs': tlg_files,
            'Properties': logged_values
        }
        
        farm_name = list(farms.values())[0] if farms else "Unknown Farm"
        report_data[farm_name][year][field_name].append(task_info)

    # --- 6. Print Report ---
    print(f"--- Agricultural Task Report ---\n")
    for farm, years in report_data.items():
        print(f"FARM: {farm}")
        for year in sorted(years.keys()):
            print(f"\n  YEAR: {year}")
            for field in sorted(years[year].keys()):
                print(f"    FIELD: {field}")
                for task in years[year][field]:
                    print(f"      TASK: {task['TaskID']}")
                    print(f"        Time: {task['TimeRange']}")
                    print(f"        Crop: {task['Crop']}")
                    print(f"        Machine: {task['Machine']}")
                    tlg_str = ', '.join(task['TLGs']) if task['TLGs'] else "None"
                    print(f"        TLG Files: {tlg_str}")
                    
                    if task['Properties']:
                        print(f"        Registered Properties (Totals):")
                        for prop in task['Properties']:
                            print(f"          - {prop}")
                    print("")

if __name__ == "__main__":
    parse_isoxml_taskdata(xml_path)

NameError: name 'xml_path' is not defined

In [8]:
xml_path = PROJECT_ROOT / 'data' / 'taskdata' / 'TASKDATA.XML'


In [9]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from datetime import datetime

def format_iso_time(iso_str):
    """Converts ISO 8601 string to human-readable 'YYYY-MM-DD HH:MM:SS'"""
    if not iso_str:
        return "Unknown"
    try:
        # Handle string slicing for simple conversion to avoid timezone library complexity
        # Expecting format like: 2023-08-16T12:39:28.000+02:00
        dt_str = iso_str.split('.')[0] # Remove milliseconds and timezone for display
        return dt_str.replace('T', ' ')
    except:
        return iso_str

def parse_isoxml_taskdata(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # --- 1. Extract Farm & Customer ---
    farms = {}
    for frm in root.findall('FRM'):
        farms[frm.attrib.get('A')] = frm.attrib.get('B', 'Unknown Farm')

    # --- 2. Extract Fields (PFD) ---
    fields = {}
    for pfd in root.findall('PFD'):
        pfd_id = pfd.attrib.get('A')
        name = pfd.attrib.get('C') or pfd.attrib.get('B') or f"Field {pfd_id}"
        fields[pfd_id] = name

    # --- 3. Extract Products/Crops (PDT) ---
    products = {}
    for pdt in root.findall('PDT'):
        products[pdt.attrib.get('A')] = pdt.attrib.get('B', 'Unknown Product')

    # --- 4. Extract Device Data (DVC, DET, DPD, DVP) ---
    # Map DET -> DVC, and store DPD/DVP definitions
    det_to_dvc = {}
    dvc_info = {}

    for dvc in root.findall('DVC'):
        dvc_id = dvc.attrib.get('A')
        dvc_name = dvc.attrib.get('B', 'Unknown Machine')
        
        for det in dvc.findall('.//DET'):
            det_to_dvc[det.attrib.get('A')] = dvc_id

        # Parse DVP (Value Presentation)
        dvp_map = {}
        for dvp in dvc.findall('.//DVP'):
            try:
                dvp_id = dvp.attrib.get('A')
                dvp_map[dvp_id] = {
                    'scale': float(dvp.attrib.get('C', 1)),
                    'offset': float(dvp.attrib.get('B', 0)),
                    'unit': dvp.attrib.get('E', '')
                }
            except: continue

        # Parse DPD (Process Data)
        dpd_map = {}
        for dpd in dvc.findall('.//DPD'):
            ddi = dpd.attrib.get('B')
            prop_name = dpd.attrib.get('E', f"DDI_{ddi}")
            dvp_ref = dpd.attrib.get('F')
            dpd_map[ddi] = {'name': prop_name, 'dvp': dvp_map.get(dvp_ref)}
            
        dvc_info[dvc_id] = {'name': dvc_name, 'dpds': dpd_map}

    # --- 5. Process Tasks (TSK) ---
    report_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    for tsk in root.findall('TSK'):
        tsk_id = tsk.attrib.get('A')
        field_ref = tsk.attrib.get('E')
        
        field_name = fields.get(field_ref, f"Unknown Field ({field_ref})")

        # Crops
        task_crops = [products[pan.attrib.get('A')] for pan in tsk.findall('PAN') if pan.attrib.get('A') in products]
        crop_name = ", ".join(task_crops) if task_crops else "None"

        # Machine
        dan = tsk.find('DAN')
        machine_name = "Generic Machine"
        dvc_id_for_task = None
        if dan is not None:
            dvc_id_for_task = dan.attrib.get('C')
            if dvc_id_for_task in dvc_info:
                machine_name = dvc_info[dvc_id_for_task]['name']

        # TLG Files
        tlg_files = [tlg.attrib.get('A') for tlg in tsk.findall('TLG')]

        # --- Process Time & Values ---
        start_times = []
        end_times = []
        logged_values = []
        
        for tim in tsk.findall('TIM'):
            if 'A' in tim.attrib: start_times.append(tim.attrib['A'])
            if 'B' in tim.attrib: end_times.append(tim.attrib['B'])

            for dlv in tim.findall('DLV'):
                ddi = dlv.attrib.get('A')
                raw_value = int(dlv.attrib.get('B', 0))
                det_ref = dlv.attrib.get('C')
                
                # Default property info
                prop_name = f"DDI {ddi}"
                final_val = raw_value
                unit = ""
                
                # 1. Check Hardcoded Overrides (The "User Request" Layer)
                if ddi == '0106': # Fugtighed (Moisture)
                    prop_name = "Fugtighed"
                    final_val = raw_value * 0.0001
                    unit = "%"
                elif ddi == '013B': # Tørstofindhold (Dry Matter)
                    prop_name = "Tørstofindhold"
                    final_val = raw_value * 0.0001
                    unit = "%"
                
                # 2. If not overridden, look up DVP definitions in XML
                else:
                    current_dvc_id = det_to_dvc.get(det_ref, dvc_id_for_task)
                    if current_dvc_id and current_dvc_id in dvc_info:
                        dvc_defs = dvc_info[current_dvc_id]['dpds']
                        if ddi in dvc_defs:
                            def_info = dvc_defs[ddi]
                            prop_name = def_info['name']
                            if def_info['dvp']:
                                dvp = def_info['dvp']
                                final_val = (raw_value * dvp['scale']) + dvp['offset']
                                unit = dvp['unit']
                            # Standard fallback units if XML is missing DVP
                            elif ddi == '0074': unit = "m²"
                            elif ddi == '005A': unit = "kg"

                # Formatting
                if isinstance(final_val, float):
                    val_str = f"{final_val:,.2f}"
                else:
                    val_str = str(final_val)

                entry = f"{prop_name}: {val_str} {unit}"
                if entry not in logged_values:
                    logged_values.append(entry)

        # Determine overall start/end
        start_ts = min(start_times) if start_times else None
        end_ts = max(end_times) if end_times else None
        
        # Determine Year
        year = start_ts[:4] if start_ts else "Unknown Year"
        
        # Formatting Time
        time_range = f"{format_iso_time(start_ts)} - {format_iso_time(end_ts)}"

        task_info = {
            'TaskID': tsk_id,
            'Machine': machine_name,
            'Crop': crop_name,
            'TimeRange': time_range,
            'TLGs': tlg_files,
            'Properties': logged_values
        }
        
        farm_name = list(farms.values())[0] if farms else "Unknown Farm"
        report_data[farm_name][year][field_name].append(task_info)

    # --- 6. Print Report ---
    print(f"--- Agricultural Task Report ---\n")
    for farm, years in report_data.items():
        print(f"FARM: {farm}")
        for year in sorted(years.keys()):
            print(f"\n  YEAR: {year}")
            for field in sorted(years[year].keys()):
                print(f"    FIELD: {field}")
                for task in years[year][field]:
                    print(f"      TASK: {task['TaskID']}")
                    print(f"        Time: {task['TimeRange']}")
                    print(f"        Crop: {task['Crop']}")
                    print(f"        Machine: {task['Machine']}")
                    tlg_str = ', '.join(task['TLGs']) if task['TLGs'] else "None"
                    print(f"        TLG Files: {tlg_str}")
                    
                    if task['Properties']:
                        print(f"        Registered Properties (Totals):")
                        for prop in task['Properties']:
                            print(f"          - {prop}")
                    print("")

if __name__ == "__main__":
    parse_isoxml_taskdata(xml_path)

--- Agricultural Task Report ---

FARM: Ledreborg Gods

  YEAR: 2022
    FIELD: 002-0, St.Amalienborg/Litgoth
      TASK: TSK189
        Time: 2022-08-04 18:06:44 - 2022-08-13 22:30:35
        Crop: HV - Hvede
        Machine: Nr: 223 C8600722
        TLG Files: TLG00123
        Registered Properties (Totals):
          - Bearb. areal: 125889 m²
          - Varighed i alt: 12917 
          - Udbytte samlet masse: 98773 kg
          - Fugtighed: 13.12 %
          - Tørstofindhold: 86.88 %

      TASK: TSK190
        Time: 2022-08-04 18:06:44 - 2022-08-13 22:30:35
        Crop: HV - Hvede
        Machine: Nr: 220  C8600702
        TLG Files: TLG00124
        Registered Properties (Totals):
          - Bearb. areal: 74253 m²
          - Varighed i alt: 7273 
          - Udbytte samlet masse: 60713 kg
          - Fugtighed: 13.12 %
          - Tørstofindhold: 86.88 %

      TASK: TSK191
        Time: 2022-08-04 18:06:44 - 2022-08-13 22:30:35
        Crop: HV - Hvede
        Machine: Nr: 22